In [1]:
from utils import PkdbModel, caffeine_idx,caffeine_idx_n, measurement_type_data, filter_out, convert_unit, unstring, to_numeric
from collections import namedtuple
import pandas as pd

{'token': 'a94a309b0392aff5ed02b52a25ba44d7abd41aa6'}


In [2]:
from IPython.display import display, HTML

def multi_table(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell    '''
    return HTML(
         '<table><tr style="background-color:white;">' +
         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
         '</tr></table>'
     )

In [3]:
%load_ext autoreload
%autoreload 2

In [11]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.to_numeric()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_caffeine = all_complete.data[caffeine_idx(all_complete.data)]


{'token': 'a94a309b0392aff5ed02b52a25ba44d7abd41aa6'}


In [12]:
Inference = namedtuple("Inference",["name","filter_out","measurement_type"])

In [13]:
inferences = [
    Inference(
        name="caffeine_clearance",
        filter_out= {"unit":["milliliter / meter ** 2 / minute"]},
        measurement_type = "clearance",
    ),
    Inference(
        name="caffeine_timecourse",
        filter_out= {"unit":["mole / liter"]},
        measurement_type = "concentration"
    ),
    Inference(
        name="caffeine_thalf",
        filter_out= {},
        measurement_type = "thalf"
    ), 
    Inference(
        name="caffeine_tmax",
        filter_out= {},
        measurement_type = "tmax"
    ),
    Inference(
        name="caffeine_vd",
        filter_out= {},
        measurement_type = "vd"
    ),
     Inference(
        name="caffeine_auc_inf",
        filter_out= {},
        measurement_type = "auc_inf"
    ),
     Inference(
        name="caffeine_auc_end",
        filter_out= {},
        measurement_type = "auc_end"
    ),
    Inference(
        name="caffeine_kel",
        filter_out= {},
        measurement_type = "kel"
    ),
]

In [14]:
for inference in inferences:
    print(inference.name)
    data = measurement_type_data(complete_caffeine,inference.measurement_type)
    data_class = PkdbModel(inference.name,destination="3-inferred")
    data_class.data = data
    [data_class.filter_out(k,v)for k,v in inference.filter_out.items()]
    data_class.infer_from_interventions()
    data_class.infer_from_outputs()
    data_class.save()

    t1 = pd.DataFrame(data_class.data.groupby(["unit", "unit_intervention"]).apply(len), columns=["count"])
    t2 = pd.DataFrame(data_class.data.groupby(["inferred"]).apply(len), columns=["count"])
    t3 = pd.DataFrame(data_class.data.groupby(["calculated"]).apply(len), columns=["count"])
    display(multi_table([t1,t2,t3]))

caffeine_clearance


caffeine_timecourse


caffeine_thalf


caffeine_tmax


caffeine_vd


caffeine_auc_inf


caffeine_auc_end


caffeine_kel
